In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob
import os
import re
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from IPython.display import display, HTML
from typing import List, Tuple, Union


from aind_dynamic_foraging_data_utils import nwb_utils as nu
import aind_dynamic_foraging_basic_analysis.licks.annotation as a
from aind_dynamic_foraging_basic_analysis.data_model.foraging_session import ForagingSessionData


from aind_dynamic_foraging_basic_analysis import plot_foraging_session, compute_foraging_efficiency

In [2]:
# Load single NWB

nwb = nu.load_nwb_from_filename('/root/capsule/data/foraging_nwb_bonsai/687582_2023-11-16_12-13-24.nwb')

/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.4.0 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [3]:
nwb

,start_time,stop_time,animal_response,rewarded_historyL,rewarded_historyR,delay_start_time,goCue_start_time,reward_outcome_time,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,reward_random_number_right,left_valve_open_time,right_valve_open_time,block_beta,block_min,block_max,min_reward_each_block,delay_beta,delay_min,delay_max,delay_duration,ITI_beta,ITI_min,ITI_max,ITI_duration,response_duration,reward_consumption_duration,auto_waterL,auto_waterR,laser_on_trial,laser_wavelength,laser_location,laser_1_power,laser_2_power,laser_on_probability,laser_duration,laser_condition,laser_condition_probability,laser_start,laser_start_offset,laser_end,laser_end_offset,laser_protocol,laser_frequency,laser_rampingdown,laser_pulse_duration,session_wide_control,fraction_of_session,session_start_with,session_alternation,auto_train_engaged,auto_train_curriculum_name,auto_train_curriculum_version,auto_train_curriculum_schema_version,auto_train_stage,auto_train_stage_overridden,lickspout_position_x,lickspout_position_y,lickspout_position_z,reward_size_left,reward_size_right
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4.489577e+06,4.489585e+06,2.0,False,False,4.489578e+06,4.489579e+06,4.489582e+06,False,False,0.8,0.711111,0.088889,0.963379,0.207240,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.004837,2.0,1.0,7.0,1.092118,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
1,4.489585e+06,4.489594e+06,2.0,False,False,4.489587e+06,4.489588e+06,4.489591e+06,True,False,0.8,0.711111,0.088889,0.526161,0.362175,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.500000,2.0,1.0,7.0,1.675733,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
2,4.489594e+06,4.489605e+06,2.0,False,False,4.489597e+06,4.489599e+06,4.489602e+06,True,False,0.8,0.711111,0.088889,0.904736,0.315749,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.500000,2.0,1.0,7.0,3.099859,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
3,4.489605e+06,4.489616e+06,2.0,False,False,4.489609e+06,4.489610e+06,4.489613e+06,True,False,0.8,0.711111,0.088889,0.758493,0.959065,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.024207,2.0,1.0,7.0,3.777559,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
,start_time,stop_time,animal_response,rewarded_historyL,rewarded_historyR,delay_start_time,goCue_start_time,reward_outcome_time,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,reward_random_number_right,left_valve_open_time,right_valve_open_time,block_beta,block_min,block_max,min_reward_each_block,delay_beta,delay_min,delay_max,delay_duration,ITI_beta,ITI_min,ITI_max,ITI_duration,response_duration,reward_consumption_duration,auto_waterL,auto_waterR,laser_on_trial,laser_wavelength,laser_location,laser_1_power,laser_2_power,laser_on_probability,laser_duration,laser_condition,laser_condition_probability,laser_start,laser_start_offset,laser_end,laser_end_offset,laser_protocol,laser_frequency,laser_rampingdown,laser_pulse_duration,session_wide_control,fraction_of_session,session_start_with,session_alternation,auto_train_engaged,auto_train_curriculum_name,auto_train_curriculum_version,auto_train_curriculum_schema_version,auto_train_stage,auto_train_stage_overridden,lickspout_position_x,lickspout_position_y,lickspout_position_z,reward_size_left,reward_size_right
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4.489577e+06,4.489585e+06,2.0,False,False,4.489578e+06,4.489579e+06,4.489582e+06,False,False,0.8,0.711111,0.088889,0.963379,0.207240,0.042,0.045,20.0,2

In [4]:
print(type(nwb.trials))  # Check if it's a tuple or a container
print(nwb.trials)  # Inspect the contents to find where the table is

<class 'pynwb.epoch.TimeIntervals'>
trials pynwb.epoch.TimeIntervals at 0x139876949069680
Fields:
  colnames: ['start_time' 'stop_time' 'animal_response' 'rewarded_historyL'
 'rewarded_historyR' 'delay_start_time' 'goCue_start_time'
 'reward_outcome_time' 'bait_left' 'bait_right'
 'base_reward_probability_sum' 'reward_probabilityL' 'reward_probabilityR'
 'reward_random_number_left' 'reward_random_number_right'
 'left_valve_open_time' 'right_valve_open_time' 'block_beta' 'block_min'
 'block_max' 'min_reward_each_block' 'delay_beta' 'delay_min' 'delay_max'
 'delay_duration' 'ITI_beta' 'ITI_min' 'ITI_max' 'ITI_duration'
 'response_duration' 'reward_consumption_duration' 'auto_waterL'
 'auto_waterR' 'laser_on_trial' 'laser_wavelength' 'laser_location'
 'laser_1_power' 'laser_2_power' 'laser_on_probability' 'laser_duration'
 'laser_condition' 'laser_condition_probability' 'laser_start'
 'laser_start_offset' 'laser_end' 'laser_end_offset' 'laser_protocol'
 'laser_frequency' 'laser_rampingdow

In [5]:
# Create df_trial

nwb.df_trials = nu.create_df_trials(nwb)

Timestamps are adjusted such that `_in_session` timestamps start at the first go cue


/opt/conda/lib/python3.9/site-packages/aind_dynamic_foraging_data_utils/nwb_utils.py:457: UserWarning: Reward before choice time. This is likely due to manual rewards
  warnings.warn("Reward before choice time. This is likely due to manual rewards")


In [7]:

def compute_foraging_efficiency(
    baited: bool,
    choice_history: Union[List, np.ndarray],
    reward_history: Union[List, np.ndarray],
    p_reward: Union[List, np.ndarray],
    autowater_offered: Union[List, np.ndarray] = None,
    random_number: Union[List, np.ndarray] = None,
) -> Tuple[float, float]:
    """Compute foraging efficiency for baited or non-baited 2-arm bandit task.

    Definition of foraging efficiency (i.e., a single value measurement to quantify "performance")
    is a complex and unsolved topic especially in the context of reward baiting. I have a
    presentation for this (https://alleninstitute.sharepoint.com/:p:/s/NeuralDynamics/EejfBIEvFA
    5DjmfOZV8atWgBx7q68GsKnavkVrfghL9y8g?e=OnR5r4).

    Simply speaking, foraging eff = actual reward of the mouse / reward of an optimal_forager in
    the same session. The question is how to define the optimal_forager.

    1. For the coupled-block-with-baiting task (Jeremiah's 2019 Neuron paper),
    I assume the optimal_forager knows the underlying reward probability and the baiting dynamics,
    and do the optimal choice pattern ("fix-and-sample" in this case, see references on p.24 of my
    slides).

    2. For the non-baiting task (Cooper Grossman), I assume the optimal_forager knows the underlying
    probability and makes the greedy choice, i.e., always choose the better side.

    This might not be the best way because the optimal_foragers I assumed is kind of cheating
    in the sense that they already know the underlying probability, but it sets an upper bound for
    all realistic agents, at least in an average sense.

    For a single session, however, there are chances where the efficiency
    can be larger than 1 because of the randomness of the task (sometimes the mice are really
    lucky that they get more reward than performing "optimally"). To **partially** alleviate this
    fluctuation, when the user provides the actual random_number that were generated in that
    session, I calculate the efficiency based on simulating the optimal_forager's performance with
    the same set of random numbers, yielding the foraging_efficiency_actual_random_seed

    Parameters
    ----------
    baited : bool
        Whether the task is baited or not.
    choice_history : Union[List, np.ndarray]
        Choice history (0 = left choice, 1 = right choice, np.nan = ignored).
        Notes: choice_history allows ignored trials or autowater trials,
           but we'll remove them in the foraging efficiency calculation.
    reward_history : Union[List, np.ndarray]
        Reward history (0 = unrewarded, 1 = rewarded).
    p_reward : Union[List, np.ndarray]
        Reward probability for both sides. The size should be (2, len(choice_history)).
    autowater_offered: Union[List, np.ndarray], optional
        If not None, indicates trials where autowater was offered, which will be excluded from
        the foraging efficiency calculation.
    random_number : Union[List, np.ndarray], optional
        The actual random numbers generated in the session (see above). Must be the same shape as
        reward_probability, by default None.

    Returns
    -------
    Tuple[float, float]
        foraging_efficiency, foraging_efficiency_actual_random_seed
    """

    # Choose the optimal_forager function based on baiting
    if baited:
        reward_optimal_func = _reward_optimal_forager_baiting
    else:
        reward_optimal_func = _reward_optimal_forager_no_baiting

    # Formatting and sanity checks
    data = ForagingSessionData(
        choice_history=choice_history,
        reward_history=reward_history,
        p_reward=p_reward,
        random_number=random_number,
        autowater_offered=autowater_offered,
    )

    # Foraging_efficiency is calculated only on finished AND non-autowater trials
    ignored = np.isnan(data.choice_history)
    valid_trials = (~ignored & ~autowater_offered) if autowater_offered is not None else ~ignored

    choice_history = data.choice_history[valid_trials]
    reward_history = data.reward_history[valid_trials]
    p_reward = data.p_reward[:, valid_trials]
    random_number = data.random_number[:, valid_trials] if data.random_number is not None else None

    # Compute reward of the optimal forager
    reward_optimal, reward_optimal_random_seed = reward_optimal_func(
        p_Ls=p_reward[0],
        p_Rs=p_reward[1],
        random_number_L=(random_number[0] if random_number is not None else None),
        random_number_R=(random_number[1] if random_number is not None else None),
    )
    reward_actual = reward_history.sum()

    return (
        reward_actual / reward_optimal,
        reward_actual / reward_optimal_random_seed,
    )


def _reward_optimal_forager_no_baiting(p_Ls, p_Rs, random_number_L, random_number_R):
    """Compute the expected reward of the optimal forager for the non-baiting task."""

    # --- Optimal-aver (use optimal expectation as 100% efficiency) ---
    reward_optimal = np.nanmean(np.max([p_Ls, p_Rs], axis=0)) * len(p_Ls)

    if random_number_L is None:
        return reward_optimal, np.nan

    # --- Optimal-actual (uses the actual random numbers by simulation)
    reward_refills = np.vstack([p_Ls >= random_number_L, p_Rs >= random_number_R])
    # Greedy choice, assuming the agent knows the groundtruth
    optimal_choices = np.argmax([p_Ls, p_Rs], axis=0)
    reward_optimal_random_seed = (
        reward_refills[0][optimal_choices == 0].sum()
        + reward_refills[1][optimal_choices == 1].sum()
    )

    return reward_optimal, reward_optimal_random_seed


def _reward_optimal_forager_baiting(p_Ls, p_Rs, random_number_L, random_number_R):
    """Compute the expected reward of the optimal forager for the baiting task."""

    # --- Optimal-aver (use optimal expectation as 100% efficiency) ---
    p_stars = np.zeros_like(p_Ls)
    for i, (p_L, p_R) in enumerate(zip(p_Ls, p_Rs)):  # Sum over all ps
        p_max = np.max([p_L, p_R])
        p_min = np.min([p_L, p_R])
        if p_min == 0 or p_max >= 1:
            p_stars[i] = p_max
        else:
            m_star = np.floor(np.log(1 - p_max) / np.log(1 - p_min))
            p_stars[i] = p_max + (1 - (1 - p_min) ** (m_star + 1) - p_max**2) / (m_star + 1)

    reward_optimal = np.nanmean(p_stars) * len(p_Ls)

    if random_number_L is None:
        return reward_optimal, np.nan

    # --- Optimal-actual (uses the actual random numbers by simulation)
    block_start_ind_left = np.where(np.diff(np.hstack([np.inf, p_Ls, np.inf])))[0].tolist()
    block_start_ind_right = np.where(np.diff(np.hstack([np.inf, p_Rs, np.inf])))[0].tolist()
    block_start_ind_effective = np.sort(
        np.unique(np.hstack([block_start_ind_left, block_start_ind_right]))
    )

    reward_refills = [p_Ls >= random_number_L, p_Rs >= random_number_R]
    reward_optimal_random_seed = 0

    # Generate optimal choice pattern
    for b_start, b_end in zip(block_start_ind_effective[:-1], block_start_ind_effective[1:]):
        p_max = np.max([p_Ls[b_start], p_Rs[b_start]])
        p_min = np.min([p_Ls[b_start], p_Rs[b_start]])
        side_max = np.argmax([p_Ls[b_start], p_Rs[b_start]])

        # Get optimal choice pattern and expected optimal rate
        if p_min == 0 or p_max >= 1:
            # Greedy is obviously optimal
            this_choice = np.array([1] * (b_end - b_start))
        else:
            m_star = np.floor(np.log(1 - p_max) / np.log(1 - p_min))
            this_choice = np.array(
                (([1] * int(m_star) + [0]) * (1 + int((b_end - b_start) / (m_star + 1))))[
                    : b_end - b_start
                ]
            )

        # Do simulation, using optimal choice pattern and actual random numbers
        reward_refill = np.vstack(
            [
                reward_refills[1 - side_max][b_start:b_end],
                reward_refills[side_max][b_start:b_end],
            ]
        ).astype(
            int
        )  # Max = 1, Min = 0
        reward_remain = [0, 0]
        for t in range(b_end - b_start):
            reward_available = reward_remain | reward_refill[:, t]
            reward_optimal_random_seed += reward_available[this_choice[t]]
            reward_remain = reward_available.copy()
            reward_remain[this_choice[t]] = 0

    return reward_optimal, (reward_optimal_random_seed if reward_optimal_random_seed else np.nan)

In [8]:
LEFT, RIGHT, IGNORE = 0, 1, 2

In [60]:
"""
    Tools for computing trial by trial metrics
    df_trials = compute_all_trial_metrics(nwb)

"""

import numpy as np

# TODO, we might want to make these parameters metric specific
WIN_DUR = 15
MIN_EVENTS = 2

LEFT, RIGHT, IGNORE = 0, 1, 2


def compute_trial_metrics(nwb):
    """
    Computes all trial by trial metrics

    response_rate,          fraction of trials with a response
    gocue_reward_rate,      fraction of trials with a reward
    response_reward_rate,   fraction of trials with a reward,
                            computed only on trials with a response
    choose_right_rate,      fraction of trials where chose right,
                            computed only on trials with a response

    reward columns,         boolean: trials where reward reward is
                            given, autowater and non_autowater trials
                            are included
    lick columns,           duration, delay, and iti periods in session,
                            right, left, and total lick counts, intertrial
                            choices: switches, consistency, and reaction time

    """
    if not hasattr(nwb, "df_trials"):
        print("You need to compute df_trials: nwb_utils.create_trials_df(nwb)")
        return

    df_trials = nwb.df_trials.copy()

    # --- Add reward-related columns ---
    df_trials['reward'] = False
    df_trials.loc[
        (df_trials.rewarded_historyL |
         df_trials.rewarded_historyR |
         df_trials.auto_waterR |
         df_trials.auto_waterL) &
        (df_trials.animal_response != IGNORE),
        'reward'
    ] = True

    df_trials['reward_non_autowater'] = False
    df_trials.loc[
        (df_trials.rewarded_historyL |
         df_trials.rewarded_historyR), 'reward_non_autowater'] = True

    df_trials['non_autowater_trial'] = False
    df_trials.loc[(df_trials.auto_waterL == 0) &
                  (df_trials.auto_waterR == 0), 'non_autowater_trial'] = True

    df_trials['non_autowater_finished_trial'] = (
        df_trials['non_autowater_trial'] &
        (df_trials['animal_response'] != IGNORE)
    )
    df_trials['ignored_non_autowater'] = (
        df_trials['non_autowater_trial'] &
        (df_trials['animal_response'] == IGNORE)
    )
    df_trials['ignored_autowater'] = (
        ~df_trials['non_autowater_trial'] &
        (df_trials['animal_response'] == IGNORE)
    )

    # --- Lick-related stats ---
    all_left_licks = nwb.acquisition['left_lick_time'].timestamps[:]
    all_right_licks = nwb.acquisition['right_lick_time'].timestamps[:]

    # Define the start and stop time for each epoch
    # Using _in_session columns
    lick_stats_epochs = {
        'gocue_stop': ['goCue_start_time_in_session', 'stop_time_in_session'],
        'delay_period': ['delay_start_time_in_session', 'goCue_start_time_in_session'],
        'iti': ['start_time_in_session', 'delay_start_time_in_session'],
    }

    # Trial-by-trial counts
    for i in range(len(df_trials)):
        for epoch_name, (start_time_name, stop_time_name) in lick_stats_epochs.items():
            start_time, stop_time = df_trials.loc[i, [start_time_name, stop_time_name]]

            # Lick analysis for the specific epoch
            left_licks = all_left_licks[(all_left_licks > start_time) &
                                        (all_left_licks < stop_time)]
            right_licks = all_right_licks[(all_right_licks > start_time) &
                                          (all_right_licks < stop_time)]
            all_licks = np.hstack([left_licks, right_licks])

            # Lick counts
            df_trials.loc[i, f'duration_{epoch_name}'] = stop_time - start_time
            df_trials.loc[i, f'n_lick_left_{epoch_name}'] = len(left_licks)
            df_trials.loc[i, f'n_lick_right_{epoch_name}'] = len(right_licks)
            df_trials.loc[i, f'n_lick_all_{epoch_name}'] = len(all_licks)

            # Lick switches
            if len(all_licks) > 1:
                _lick_identity = np.hstack([np.ones(len(left_licks)) *
                                            LEFT, np.ones(len(right_licks)) * RIGHT])
                _lick_identity_sorted = [x for x, _ in sorted
                                         (zip(_lick_identity, all_licks),
                                          key=lambda pairs: pairs[1])]
                df_trials.loc[i, f'n_lick_switches_{epoch_name}'] = (
                    np.sum(np.diff(_lick_identity_sorted) != 0)
                )

                # Lick consistency
                choice = df_trials.loc[i, 'animal_response']
                df_trials.loc[i, f'n_lick_consistency_{epoch_name}'] = (
                    np.sum(_lick_identity_sorted == choice) / len(_lick_identity_sorted)
                    if len(_lick_identity_sorted) > 0 else np.nan
                )
            else:
                df_trials.loc[i, f'n_lick_switches_{epoch_name}'] = 0
                df_trials.loc[i, f'n_lick_consistency_{epoch_name}'] = np.nan

            # Special treatment for gocue to stop epoch
            if epoch_name == 'gocue_stop':
                # Reaction time
                first_lick = all_licks.min() if len(all_licks) > 0 else np.nan
                df_trials.loc[i, 'reaction_time'] = (
                    first_lick - df_trials.loc[i, 'goCue_start_time_in_session']
                    if not np.isnan(first_lick) else np.nan
                )

                # Handle ignored trials
                if df_trials.loc[i, 'animal_response'] == IGNORE:
                    df_trials.loc[i, 'reaction_time'] = np.nan
                    df_trials.loc[i, 'n_valid_licks_left'] = 0
                    df_trials.loc[i, 'n_valid_licks_right'] = 0
                    df_trials.loc[i, 'n_valid_licks_all'] = 0

    df_trials["RESPONDED"] = [x in [0, 1] for x in df_trials["animal_response"].values]
    # Rolling fraction of goCues with a response
    df_trials["response_rate"] = (
        df_trials["RESPONDED"].rolling(WIN_DUR, min_periods=MIN_EVENTS, center=True).mean()
    )

    # Rolling fraction of goCues with a response
    df_trials["gocue_reward_rate"] = (
        df_trials["earned_reward"].rolling(WIN_DUR, min_periods=MIN_EVENTS, center=True).mean()
    )

    # Rolling fraction of responses with a response
    df_trials["RESPONSE_REWARD"] = [
        x[0] if x[1] else np.nan for x in zip(df_trials["earned_reward"], df_trials["RESPONDED"])
    ]
    df_trials["response_reward_rate"] = (
        df_trials["RESPONSE_REWARD"].rolling(WIN_DUR, min_periods=MIN_EVENTS, center=True).mean()
    )

    # Rolling fraction of choosing right
    df_trials["WENT_RIGHT"] = [x if x in [0, 1] else np.nan for x in df_trials["animal_response"]]
    df_trials["choose_right_rate"] = (
        df_trials["WENT_RIGHT"].rolling(WIN_DUR, min_periods=MIN_EVENTS, center=True).mean()
    )

    # Clean up temp columns
    drop_cols = [
        "RESPONDED",
        "RESPONSE_REWARD",
        "WENT_RIGHT",
    ]
    df_trials = df_trials.drop(columns=drop_cols)

    return df_trials


In [61]:
nwb.df_trials = compute_trial_metrics(nwb)

In [62]:
def compute_session_metadata(nwb):
    """
    block structure metrics,        block, contrast, and effective
                                    probability metrics
    duration metrics,               gocue, delay period, and iti
                                    metrics
    reward size metrics,            left and right reward volumes
    lick spout movement,            movement of lickspout during session:
                                    range, initial position, median position
    autotrain categories,           curriculum version, name, schema,
                                    current_stage_actual, and if overriden
    """
    if not hasattr(nwb, 'df_trials'):
        print('You need to compute df_trials: nwb_utils.create_trials_df(nwb)')
        return

    df_trials = nwb.df_trials.copy()

    # Block information
    def _get_block_starts(p_L, p_R):
        """Find the indices of block starts"""
        block_start_ind_left = np.where(np.hstack([True, np.diff(p_L) != 0]))[0]
        block_start_ind_right = np.where(np.hstack([True, np.diff(p_R) != 0]))[0]
        block_start_ind_effective = np.sort(
            np.unique(
                np.hstack([block_start_ind_left, block_start_ind_right])
            )
        )
        return block_start_ind_left, block_start_ind_right, block_start_ind_effective

    # -- Key meta data --
    session_start_time = nwb.session_start_time
    session_date = session_start_time.strftime("%Y-%m-%d")
    subject_id = nwb.subject.subject_id

    # -- Block and probability analysis --
    p_L = df_trials.reward_probabilityL.values
    p_R = df_trials.reward_probabilityR.values
    p_contrast = np.max([p_L, p_R], axis=0) / (np.min([p_L, p_R], axis=0) + 1e-6)
    p_contrast[p_contrast > 100] = 100  # Cap the contrast at 100

    # Parse effective block
    block_start_left, block_start_right, block_start_effective = _get_block_starts(p_L, p_R)
    if 'uncoupled' not in nwb.protocol.lower():
        if not (len(block_start_left) == len(block_start_right)
                and all(block_start_left == block_start_right)):
            logger.warning("Blocks are not fully aligned in a Coupled task!")

    # -- Metadata dictionary --
    dict_meta = {
        'subject_id': subject_id,
        'session_date': session_date,
        'user_name': nwb.experimenter[0],
        'task': nwb.protocol,
        'session_start_time': session_start_time,


        # Block structure metrics
        'p_reward_sum_mean': np.mean(p_L + p_R),
        'p_reward_sum_std': np.std(p_L + p_R),
        'p_reward_sum_median': np.median(p_L + p_R),

        'p_reward_contrast_mean': np.mean(p_contrast),
        'p_reware_contrast_median': np.median(p_contrast),

        'effective_block_length_mean': np.mean(np.diff(block_start_effective)),
        'effective_block_length_std': np.std(np.diff(block_start_effective)),
        'effective_block_length_median': np.median(np.diff(block_start_effective)),
        'effective_block_length_min': np.min(np.diff(block_start_effective)),
        'effective_block_length_max': np.max(np.diff(block_start_effective)),

        # Duration metrics
        'duration_gocue_stop_mean': df_trials.loc[:, 'duration_gocue_stop'].mean(),
        'duration_gocue_stop_std': df_trials.loc[:, 'duration_gocue_stop'].std(),
        'duration_gocue_stop_median': df_trials.loc[:, 'duration_gocue_stop'].median(),
        'duration_gocue_stop_min': df_trials.loc[:, 'duration_gocue_stop'].min(),
        'duration_gocue_stop_max': df_trials.loc[:, 'duration_gocue_stop'].max(),

        'duration_delay_period_mean': df_trials.loc[:, 'duration_delay_period'].mean(),
        'duration_delay_period_std': df_trials.loc[:, 'duration_delay_period'].std(),
        'duration_delay_period_median': df_trials.loc[:, 'duration_delay_period'].median(),
        'duration_delay_period_min': df_trials.loc[:, 'duration_delay_period'].min(),
        'duration_delay_period_max': df_trials.loc[:, 'duration_delay_period'].max(),

        'duration_iti_mean': df_trials.loc[:, 'duration_iti'].mean(),
        'duration_iti_std': df_trials.loc[:, 'duration_iti'].std(),
        'duration_iti_median': df_trials.loc[:, 'duration_iti'].median(),
        'duration_iti_min': df_trials.loc[:, 'duration_iti'].min(),
        'duration_iti_max': df_trials.loc[:, 'duration_iti'].max(),

        # Reward size metrics
        'reward_volume_left_mean': df_trials.loc[df_trials.reward, 'reward_size_left'].mean(),
        'reward_volume_right_mean': df_trials.loc[df_trials.reward, 'reward_size_right'].mean(),

        # Lickspouts movement range (in um)
        **{f'lickspout_movement_range_{axis}':
            np.ptp(df_trials[f'lickspout_position_{axis}']) for axis in 'xyz'},
        **{f'lickspout_initial_pos_{axis}':
            df_trials[f'lickspout_position_{axis}'][0] for axis in 'xyz'},
        **{f'lickspout_median_pos_{axis}':
            np.median(df_trials[f'lickspout_position_{axis}']) for axis in 'xyz'},
    }

    # Add flag for old bpod session
    if 'bpod' in nwb.session_description:
        dict_meta['old_bpod_session'] = True

    # Create metadata DataFrame
    df_session_meta = pd.DataFrame(dict_meta, index=[0])

    # Add automatic training info
    if 'auto_train_engaged' in df_trials.columns:
        df_session_meta['auto_train', 'curriculum_name'] = (
            np.nan
            if df_trials.auto_train_curriculum_name.mode()[0].lower() == 'none'
            else df_trials.auto_train_curriculum_name.mode()[0]
        )
        df_session_meta['auto_train', 'curriculum_version'] = (
            np.nan
            if df_trials.auto_train_curriculum_version.mode()[0].lower() == 'none'
            else df_trials.auto_train_curriculum_version.mode()[0]
        )
        df_session_meta['auto_train', 'curriculum_schema_version'] = (
            np.nan
            if df_trials.auto_train_curriculum_schema_version.mode()[0].lower() == 'none'
            else df_trials.auto_train_curriculum_schema_version.mode()[0]
        )
        df_session_meta['auto_train', 'current_stage_actual'] = (
            np.nan
            if df_trials.auto_train_stage.mode()[0].lower() == 'none'
            else df_trials.auto_train_stage.mode()[0]
        )
        df_session_meta['auto_train', 'if_overriden_by_trainer'] = (
            np.nan
            if all(df_trials.auto_train_stage_overridden.isna())
            else df_trials.auto_train_stage_overridden.mode()[0]
        )
        # Check consistency of auto train settings
        df_session_meta['auto_train', 'if_consistent_within_session'] = len(df_trials.groupby(
            [col for col in df_trials.columns if 'auto_train' in col]
        )) == 1
    else:
        for field in [
            'curriculum_name',
            'curriculum_version',
            'curriculum_schema_version',
            'current_stage_actual',
            'if_overriden_by_trainer'
        ]:
            df_session_meta['auto_train', field] = None

    return df_session_meta


def compute_session_metrics(nwb):
    """
    Compute all session metadata and performance metrics

    basic performance metrics,      both autowater and non-autowater specific
                                    rates (total, finished, ignored, finished rate,
                                    ignored rate, reward rate)
    calculated metrics,             foraging efficiency, foraging performance
                                    (both normal and random seed), bias naive,
                                    chosen probability
    lick metrics,                   reaction mean and median, early lick rate, invalid
                                    lick ratio, double dipping finished rates (reward and total),
                                    lick consistency means (total, reward, and non-rewarded)

    New addition: chosen_probability - average difference between the chosen probability
    and non-chosen probability / the difference between the largest and smallest probability
    in the session
    """

    if not hasattr(nwb, 'df_trials'):
        print('You need to compute df_trials: nwb_utils.create_trials_df(nwb)')
        return

    df_trials = nwb.df_trials.copy()

    # Add session Metdata
    df_session_meta = compute_session_metadata(nwb)

    # -- Performance Metrics --
    n_total_trials = len(df_trials)
    n_finished_trials = (df_trials.animal_response != IGNORE).sum()

    # Actual foraging trials (autowater excluded)
    n_total_trials_non_autowater = df_trials.non_autowater_trial.sum()
    n_finished_trials_non_autowater = df_trials.non_autowater_finished_trial.sum()

    n_reward_trials_non_autowater = df_trials.reward_non_autowater.sum()
    reward_rate_non_autowater_finished = (
        n_reward_trials_non_autowater / n_finished_trials_non_autowater
        if n_finished_trials_non_autowater > 0
        else np.nan
    )

    # Foraging efficiency
    foraging_eff, foraging_eff_random_seed = compute_foraging_efficiency(
        baited='without bait' not in nwb.protocol.lower(),
        choice_history=df_trials.animal_response.map({0: 0, 1: 1, 2: np.nan}).values,
        reward_history=df_trials.rewarded_historyL | df_trials.rewarded_historyR,
        p_reward=[
            df_trials.reward_probabilityL.values,
            df_trials.reward_probabilityR.values,
        ],
        random_number=[
            df_trials.reward_random_number_left.values,
            df_trials.reward_random_number_right.values,
        ],
        autowater_offered=(df_trials.auto_waterL == 1) | (df_trials.auto_waterR == 1)
    )

    all_lick_number = (
        len(nwb.acquisition['left_lick_time'].timestamps) +
        len(nwb.acquisition['right_lick_time'].timestamps)
    )

    # Naive bias calculation
    n_left = ((df_trials.animal_response == LEFT) & (df_trials.non_autowater_trial)).sum()
    n_right = ((df_trials.animal_response == RIGHT) & (df_trials.non_autowater_trial)).sum()
    bias_naive = 2 * (n_right / (n_left + n_right) - 0.5) if n_left + n_right > 0 else np.nan
    finished_rate = (
        n_finished_trials_non_autowater / n_total_trials_non_autowater
        if n_total_trials_non_autowater > 0
        else np.nan
    )

    # Probability chosen calculation
    probability_chosen = []
    probability_not_chosen = []

    for _, row in df_trials.iterrows():
        if row.animal_response == 2:
            probability_chosen.append(np.nan)
            probability_not_chosen.append(np.nan)
        elif (
            row.animal_response == 0
        ):  # Chosen = left choice left probability, not chosen = left choice right probability
            probability_chosen.append(row.reward_probabilityL)
            probability_not_chosen.append(row.reward_probabilityR)
        else:  # Chosen = right choice right probability, not chosen = right choice left probability
            probability_chosen.append(row.reward_probabilityR)
            probability_not_chosen.append(row.reward_probabilityL)

    df_trials["probability_chosen"] = probability_chosen
    df_trials["probability_not_chosen"] = probability_not_chosen

    # Calculate chosen probability
    average = df_trials["probability_chosen"] - df_trials["probability_not_chosen"]

    p_larger_global = max(
        df_trials["probability_chosen"].max(), df_trials["probability_not_chosen"].max()
    )

    p_smaller_global = min(
        df_trials["probability_chosen"].min(), df_trials["probability_not_chosen"].min()
    )

    mean_difference = average.mean()
    chosen_probability = mean_difference / (p_larger_global - p_smaller_global)

    # Performance dictionary
    dict_performance = {
        # Basic performance metrics
        'total_trials_with_autowater': n_total_trials,
        'finished_trials_with_autowater': n_finished_trials,
        'finished_rate_with_autowater': n_finished_trials / n_total_trials,
        'ignore_rate_with_autowater': 1 - n_finished_trials / n_total_trials,
        'autowater_collected': (~df_trials.non_autowater_trial
                                & (df_trials.animal_response != IGNORE)).sum(),
        'autowater_ignored': (~df_trials.non_autowater_trial
                              & (df_trials.animal_response == IGNORE)).sum(),

        'total_trials': n_total_trials_non_autowater,
        'finished_trials': n_finished_trials_non_autowater,
        'ignored_trials': n_total_trials_non_autowater - n_finished_trials_non_autowater,
        'finished_rate': finished_rate,
        'ignore_rate': 1 - finished_rate,

        'reward_trials': n_reward_trials_non_autowater,
        'reward_rate': reward_rate_non_autowater_finished,
        'foraging_eff': foraging_eff,
        'foraging_eff_random_seed': foraging_eff_random_seed,

        'foraging_performance': foraging_eff * finished_rate,
        'foraging_performance_random_seed': foraging_eff_random_seed * finished_rate,

        'bias_naive': bias_naive,

        # New Metrics
        'chosen_probability': chosen_probability,

        # Lick timing metrics
        'reaction_time_median': df_trials.loc[:, 'reaction_time'].median(),
        'reaction_time_mean': df_trials.loc[:, 'reaction_time'].mean(),

        'early_lick_rate':
            (df_trials.loc[:, 'n_lick_all_delay_period'] > 0).sum() / n_total_trials,

        'invalid_lick_ratio':
            (all_lick_number - df_trials.loc[:, 'n_lick_all_gocue_stop'].sum()) / all_lick_number,

        # Lick consistency metrics
        'double_dipping_rate_finished_trials':
            (df_trials.loc[(df_trials.animal_response != IGNORE),
                           'n_lick_switches_gocue_stop'] > 0).sum()
            / (df_trials.animal_response != IGNORE).sum(),
        'double_dipping_rate_finished_reward_trials':
            (df_trials.loc[df_trials.reward, 'n_lick_switches_gocue_stop'] > 0).sum()
            / df_trials.reward.sum(),
        'double_dipping_rate_finished_noreward_trials':
            (df_trials.loc[(df_trials.animal_response != IGNORE)
                           & (~df_trials.reward), 'n_lick_switches_gocue_stop'] > 0).sum()
            / ((df_trials.animal_response != IGNORE) & (~df_trials.reward)).sum(),
        'lick_consistency_mean_finished_trials':
            df_trials.loc[(df_trials.animal_response != IGNORE),
                          'n_lick_consistency_gocue_stop'].mean(),
        'lick_consistency_mean_finished_reward_trials':
            df_trials.loc[df_trials.reward, 'n_lick_consistency_gocue_stop'].mean(),
        'lick_consistency_mean_finished_noreward_trials':
            df_trials.loc[(df_trials.animal_response != IGNORE)
                          & (~df_trials.reward), 'n_lick_consistency_gocue_stop'].mean(),
        }

    # Create performance Dataframe
    df_session_performance = pd.DataFrame(dict_performance, index=[0])

    # Create session DataFrame
    df_session = pd.concat([df_session_meta, df_session_performance], axis=1).reset_index(drop=True)

    return df_session

In [63]:
final_df = compute_session_metrics(nwb)

In [64]:
display(HTML(final_df.to_html()))


,subject_id,session_date,user_name,task,session_start_time,p_reward_sum_mean,p_reward_sum_std,p_reward_sum_median,p_reward_contrast_mean,p_reware_contrast_median,effective_block_length_mean,effective_block_length_std,effective_block_length_median,effective_block_length_min,effective_block_length_max,duration_gocue_stop_mean,duration_gocue_stop_std,duration_gocue_stop_median,duration_gocue_stop_min,duration_gocue_stop_max,duration_delay_period_mean,duration_delay_period_std,duration_delay_period_median,duration_delay_period_min,duration_delay_period_max,duration_iti_mean,duration_iti_std,duration_iti_median,duration_iti_min,duration_iti_max,reward_volume_left_mean,reward_volume_right_mean,lickspout_movement_range_x,lickspout_movement_range_y,lickspout_movement_range_z,lickspout_initial_pos_x,lickspout_initial_pos_y,lickspout_initial_pos_z,lickspout_median_pos_x,lickspout_median_pos_y,lickspout_median_pos_z,"(auto_train, curriculum_name)","(auto_train, curriculum_version)","(auto_train, curriculum_schema_version)","(auto_train, current_stage_actual)","(auto_train, if_overriden_by_trainer)","(auto_train, if_consistent_within_session)",total_trials_with_autowater,finished_trials_with_autowater,finished_rate_with_autowater,ignore_rate_with_autowater,autowater_collected,autowater_ignored,total_trials,finished_trials,ignored_trials,finished_rate,ignore_rate,reward_trials,reward_rate,foraging_eff,foraging_eff_random_seed,foraging_performance,foraging_performance_random_seed,bias_naive,chosen_probability,reaction_time_median,reaction_time_mean,early_lick_rate,invalid_lick_ratio,double_dipping_rate_finished_trials,double_dipping_rate_finished_reward_trials,double_dipping_rate_finished_noreward_trials,lick_consistency_mean_finished_trials,lick_consistency_mean_finished_reward_trials,lick_consistency_mean_finished_noreward_trials
0,687582,2023-11-16,Avalon Amaya,Coupled Baiting,2023-11-16 12:13:24.835308-08:00,0.8,2.220446e-16,0.8,7.99991,7.99991,42.75,22.635242,40.0,21,99,5.017856,1.331447,6.043264,3.038144,10.3992,1.501248,0.613286,1.38464,1.000704,6.750848,3.448449,1.865812,3.09848,1.004,7.013984,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,542,241,0.444649,0.555351,39,19,484,202,282,0.417355,0.582645,104,0.514851,0.70817,0.727273,0.295559,0.303531,0.534653,0.170124,NaN,NaN,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN


In [65]:
final_df.columns

Index([                                    'subject_id',
                                         'session_date',
                                            'user_name',
                                                 'task',
                                   'session_start_time',
                                    'p_reward_sum_mean',
                                     'p_reward_sum_std',
                                  'p_reward_sum_median',
                               'p_reward_contrast_mean',
                             'p_reware_contrast_median',
                          'effective_block_length_mean',
                           'effective_block_length_std',
                        'effective_block_length_median',
                           'effective_block_length_min',
                           'effective_block_length_max',
                             'duration_gocue_stop_mean',
                              'duration_gocue_stop_std',
                           'dur